In [52]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import roc_auc_score,log_loss
from tqdm import tqdm
from sklearn.preprocessing import  OneHotEncoder,StandardScaler
import pandas_profiling
import datetime
from catboost import CatBoostClassifier
%pylab inline
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv('data/uplift_train.csv')
df1 = pd.read_csv('data/clients.csv')
df2 = pd.read_csv('data/products.csv')
test = pd.read_csv('data/uplift_test.csv')

In [3]:
data_train = pd.merge(df1,df)
miss = data_train[data_train['first_redeem_date'].isna()]
miss['first_redeem_date']=miss['first_issue_date']
data_train.first_redeem_date[miss.index]=miss.first_redeem_date.values
data_train

,client_id,first_issue_date,first_redeem_date,age,gender,treatment_flg,target
0,000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U,0,1
1,000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F,1,1
2,00010925a5,2018-07-24 16:21:29,2018-09-14 16:12:49,83,U,1,1
3,0001f552b0,2017-06-30 19:20:38,2018-08-28 12:59:45,33,F,1,1
4,00020e7b18,2017-11-27 11:41:45,2018-01-10 17:50:05,73,U,1,1
...,...,...,...,...,...,...,...
200034,fffe0abb97,2017-11-27 08:56:54,2018-02-11 09:26:08,35,F,0,0
200035,fffe0ed719,2017-09-15 08:53:24,2017-12-12 14:50:12,69,U,0,1
200036,fffea1204c,2018-01-31 16:59:37,2018-03-12 17:02:27,73,F,0,1
200037,fffeca6d22,2017-12-28 11:56:13,2017-12-28 11:56:13,77,F,1,0


In [4]:
data_test=pd.merge(df1,test)
miss = data_test[data_test['first_redeem_date'].isna()]
miss['first_redeem_date']=miss['first_issue_date']
data_test.first_redeem_date[miss.index]=miss.first_redeem_date.values
data_test

,client_id,first_issue_date,first_redeem_date,age,gender
0,000048b7a6,2018-12-15 13:33:11,2018-12-15 13:33:11,68,F
1,000073194a,2017-05-23 12:56:14,2017-11-24 11:18:01,60,F
2,00007c7133,2017-05-22 16:17:08,2018-12-31 17:17:33,67,U
3,00007f9014,2017-08-22 13:35:01,2019-02-15 19:19:18,45,F
4,0000a90cf7,2017-05-20 12:34:50,2017-07-08 12:48:22,45,U
...,...,...,...,...,...
200118,fffeacbe3e,2017-07-19 08:38:17,2017-10-13 10:59:14,47,F
200119,fffece623e,2018-05-13 15:09:30,2018-12-27 19:06:39,67,U
200120,ffff3dfff8,2018-11-01 16:06:52,2018-12-07 17:33:47,56,F
200121,ffffaab9da,2017-08-25 18:43:24,2017-12-08 17:53:19,23,F


In [5]:
data_train['first_issue_date']=pd.to_datetime(data_train['first_issue_date'])
data_train['first_redeem_date']=pd.to_datetime(data_train['first_redeem_date'])
data_test['first_issue_date']=pd.to_datetime(data_test['first_issue_date'])
data_test['first_redeem_date']=pd.to_datetime(data_test['first_redeem_date'])

In [6]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200039 entries, 0 to 200038
Data columns (total 7 columns):
client_id            200039 non-null object
first_issue_date     200039 non-null datetime64[ns]
first_redeem_date    200039 non-null datetime64[ns]
age                  200039 non-null int64
gender               200039 non-null object
treatment_flg        200039 non-null int64
target               200039 non-null int64
dtypes: datetime64[ns](2), int64(3), object(2)
memory usage: 17.2+ MB


In [7]:
data_train['fake_age'] = data_train['age'].apply(lambda x:int((x>90)or(x<16)))
data_test['fake_age'] = data_test['age'].apply(lambda x:int((x>90)or(x<16)))

In [8]:
data_train['first_issue_date'] = data_train['first_issue_date'].apply(lambda x: x.timestamp())
data_train['first_redeem_date'] = data_train['first_redeem_date'].apply(lambda x: x.timestamp())
data_test['first_issue_date'] = data_test['first_issue_date'].apply(lambda x: x.timestamp())
data_test['first_redeem_date'] = data_test['first_redeem_date'].apply(lambda x: x.timestamp())

In [9]:
purch = pd.read_csv('data/purchases.csv',parse_dates=['transaction_datetime'])

In [10]:
user_id = purch.drop_duplicates('client_id').client_id.values

In [11]:
transactions = purch.drop_duplicates('transaction_id')

In [12]:
transactions['transaction_datetime'] = transactions['transaction_datetime'].apply(lambda x: x.timestamp())

In [13]:
#число покупок
a = transactions[['client_id','transaction_id']].values
pok=[]
user=''
count=0
for row in a:
    if row[0]!=user:
        pok.append(count)
        user =row[0]
        count=1
    else:
        count+=1
pokupki = pok[1:]
pokupki.append(count)

In [14]:
#даты последних покупок
dates = purch.drop_duplicates('client_id',keep='last').transaction_datetime.values

In [15]:
#число магазинов
b = transactions[['client_id','store_id']].values
store=[]
user=''
count=0
for row in b:
    if row[0]!=user:
        store.append(count)
        user =row[0]
        count=1
        last_store = row[1]
    else:
        if row[1]!=last_store:
            count+=1
            last_store = row[1]
stores = store[1:]
stores.append(count)

In [16]:
#cредняя сумма покупки
c = transactions[['client_id','purchase_sum']].values
avsum=[]
user=''
count=1
sred=0
for row in c:
    if row[0]!=user:
        avsum.append(sred/count)
        user =row[0]
        count=1
        sred=row[1]
    else:
        count+=1
        sred+=row[1]
average = avsum[1:]
average.append(sred/count)

In [17]:
produkti = purch.product_id.values
useri = purch.client_id.values

In [18]:
alcohol = set(df2[df2['is_alcohol']==1].product_id.values)
owner = set(df2[df2['is_own_trademark']==1].product_id.values)

In [19]:
alc = lambda x: int(x in alcohol)
is_alc = [alc(i) for i in produkti]
own = lambda x: int(x in owner)
is_own = [own(i) for i in produkti]

In [20]:
purch['product_is_alco']=is_alc
purch['product_is_own']=is_own

In [21]:
#число купленного алкоголя и собственной торговой марки
a = transactions[['client_id','transaction_id']].values
alc_count=[]
own_count=[]
user=''
count_alc=0
count_own=0
for i in range(len(useri)):
    if useri[i]!=user:
        alc_count.append(count_alc)
        own_count.append(count_own)
        user =useri[i]
        count_alc=is_alc[i]
        count_own=is_own[i]
    else:
        count_alc+=is_alc[i]
        count_own+=is_own[i]
alc_count = alc_count[1:]
own_count = own_count[1:]
alc_count.append(count_alc)
own_count.append(count_own)

In [22]:
new_features = pd.DataFrame({'client_id':user_id,'transaction_count':pokupki,'av_sum':average,
                             'last_transaction':dates,'store_count':stores,
                             'alc_count':alc_count,'own_count':own_count})

In [23]:
new_features['last_transaction'] = new_features['last_transaction'].apply(lambda x: x.timestamp())

In [24]:
data_train_full = pd.merge(data_train,new_features)
data_test_full = pd.merge(data_test,new_features)

In [25]:
data_train_full.index=data_train_full['client_id']
del data_train_full['client_id']

In [26]:
data_test_full.index=data_test_full['client_id']
del data_test_full['client_id']

In [27]:
X = data_train_full.drop(['target'],axis='columns')

In [28]:
X.columns

Index(['first_issue_date', 'first_redeem_date', 'age', 'gender',
       'treatment_flg', 'fake_age', 'transaction_count', 'av_sum',
       'last_transaction', 'store_count', 'alc_count', 'own_count'],
      dtype='object')

In [29]:
Cat_cols = ['gender','fake_age']
Num_cols = ['first_issue_date', 'first_redeem_date', 'age','transaction_count', 'av_sum',
       'last_transaction', 'store_count', 'alc_count', 'own_count']

In [30]:
enc = OneHotEncoder(sparse=False)
scaler = StandardScaler()

In [31]:
X_cat_encoded = pd.DataFrame(enc.fit_transform(X[Cat_cols]))

In [32]:
X_num_scaled = pd.DataFrame(scaler.fit_transform(X[Num_cols]),columns=Num_cols)

In [33]:
Test_cat = pd.DataFrame(enc.fit_transform(data_test_full[Cat_cols]))

In [34]:
Test_num =  pd.DataFrame(scaler.fit_transform(data_test_full[Num_cols]),columns=Num_cols)

In [55]:
test = pd.concat([Test_cat,Test_num],axis=1)

In [36]:
train =pd.concat([X_cat_encoded,X_num_scaled,data_train['treatment_flg']],axis=1)

In [41]:
train_full = pd.concat([X_cat_encoded,X_num_scaled,data_train[['treatment_flg','target']]],axis=1)

In [63]:
data_train_full

,first_issue_date,first_redeem_date,age,gender,treatment_flg,target,fake_age,transaction_count,av_sum,last_transaction,store_count,alc_count,own_count,new_target
client_id,,,,,,,,,,,,,,
000012768d,1.501948e+09,1.515094e+09,45,U,0,1,0,4,700.750000,1.552576e+09,3,0,4,0
000036f903,1.491832e+09,1.492951e+09,72,F,1,1,0,32,306.406250,1.552819e+09,14,1,14,1
00010925a5,1.532449e+09,1.536942e+09,83,U,1,1,0,18,326.833333,1.552029e+09,8,1,9,1
0001f552b0,1.498850e+09,1.535461e+09,33,F,1,1,0,15,410.345333,1.552734e+09,7,0,10,1
00020e7b18,1.511783e+09,1.515607e+09,73,U,1,1,0,18,1434.422778,1.552651e+09,7,0,44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffe0abb97,1.511773e+09,1.518341e+09,35,F,0,0,0,9,301.824444,1.552632e+09,5,0,3,1
fffe0ed719,1.505466e+09,1.513090e+09,69,U,0,1,0,30,347.249333,1.552919e+09,2,0,29,0
fffea1204c,1.517418e+09,1.520874e+09,73,F,0,1,0,17,252.771765,1.552478e+09,1,0,0,0


In [37]:
cat_params = {'learning_rate':0.01, 'max_depth':3, 'task_type':'CPU',
              'loss_function':'Logloss', 'eval_metric':'Logloss',
               'iterations':20000, 'od_type': "Iter", 'od_wait':200
}

In [44]:
def uplift_at_k(y_true, uplift, treatment, k=0.3):
    order = np.argsort(-uplift)
    treatment_n = int((treatment == 1).sum() * k)
    treatment_p = y_true[order][treatment[order] == 1][:treatment_n].mean()
    control_n = int((treatment == 0).sum() * k)
    control_p = y_true[order][treatment[order] == 0][:control_n].mean()
    score_at_k = treatment_p - control_p
    
    return score_at_k

In [46]:
def trans_train_model(df_X, df_X_test, num_folds=5, random_state=0, verbose=2):

    cat_params['random_state'] = random_state
    
    df_X['new_target'] = (df_X['target'] + df_X['treatment_flg'] + 1) % 2
    df_y = df_X[['new_target']]
    treatment = df_X['treatment_flg'].to_numpy()
    old_target = df_X['target'].to_numpy()
    df_X = df_X.drop(['target', 'new_target', 'treatment_flg'], axis=1, errors='ignore')    
    
    X = df_X.to_numpy()
    y = df_y.to_numpy()
    X_test = df_X_test.to_numpy()
    
    folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=random_state)
    scores = []
    uplift_scores = []    
    prediction = np.zeros(len(X_test))

    for i, (train_index, valid_index) in enumerate(folds.split(X, y)):    
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]        
        treat_valid = treatment[valid_index]
        old_target_vaild = old_target[valid_index]
        
        f = CatBoostClassifier(**cat_params)
        f.fit(X_train, y_train, eval_set=(X_valid, y_valid), use_best_model=True, verbose=False)
        
    
        y_pred_valid = f.predict_proba(X_valid)[:, 1]
        score = log_loss(y_valid, y_pred_valid)
        uplift_score = uplift_at_k(old_target_vaild, y_pred_valid, treat_valid)
        uplift_scores.append(uplift_score)
        if (verbose > 1):
            print('OOF Uplift score: {0:.5f}'.format(uplift_score))
        
        scores.append(score)
        # predict on test and accumulate the result
        y_pred = f.predict_proba(X_test)[:, 1]
        prediction += y_pred

    # get average prediction from all models
    prediction /= num_folds

    if (verbose > 0):
        print('CV mean score: {0:.5f}, std: {1:.5f}'.format(np.mean(scores), np.std(scores)))    
        print('Uplift score @30%: {0:.5f}, std: {1:.5f}'.format(np.mean(uplift_scores), np.std(uplift_scores)))    
        
    return prediction

In [56]:
pred = trans_train_model(train_full, test, verbose=2)

OOF Uplift score: 0.08533
OOF Uplift score: 0.08037
OOF Uplift score: 0.07851
OOF Uplift score: 0.07751
OOF Uplift score: 0.07385
CV mean score: 0.68862, std: 0.00018
Uplift score @30%: 0.07911, std: 0.00376


In [62]:
pred = trans_train_model(data_train_full,data_test_full, verbose=2)

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=3]="U": Cannot convert 'b'U'' to float

In [59]:
df_submission = pd.DataFrame({'client_id':data_test_full.index,'uplift': pred})
df_submission.to_csv('sub_4.csv', index=False)

In [57]:
data_test_full.index

Index(['000048b7a6', '000073194a', '00007c7133', '00007f9014', '0000a90cf7',
       '0000b59cec', '0000bb4e4e', '0000bcec9c', '0000eecb82', '0000f0ecdb',
       ...
       'fffb310977', 'fffbfc9346', 'fffc68800d', 'fffc6fd3ea', 'fffde49bc6',
       'fffeacbe3e', 'fffece623e', 'ffff3dfff8', 'ffffaab9da', 'ffffeb5619'],
      dtype='object', name='client_id', length=200123)

In [137]:
sol1.index = sol1['client_id']
del sol1['client_id']

In [138]:
sol1

,uplift
client_id,
000048b7a6,0.056886
000073194a,0.037388
00007c7133,0.038690
00007f9014,0.017279
0000a90cf7,0.015631
...,...
fffeacbe3e,0.033306
fffece623e,0.035032
ffff3dfff8,0.052328


In [139]:
sol1.to_csv('sol2.csv')

In [113]:
from catboost import CatBoostClassifier

In [133]:
cbc = CatBoostClassifier(thread_count=2, random_state=17, silent=True)
cbc.fit(train,data_train['target'])

In [134]:
preds1 = [i[1] for i in cbc.predict_proba(test1)]
preds0 = [i[1] for i in cbc.predict_proba(test0)]